# 03. 다중 레이블 분류 (Multi-label Classification)
### 과목: NLP Text Classification
---

## 1. "한 영화가 액션이면서 동시에 SF일 수 있을까?"
지금까지는 하나의 정답만 고르는 문제(다중 분류, Multi-class)를 풀었습니다.
하지만 현실 세계에서는 **하나의 데이터가 여러 개의 정답(태그)을 가질 수 있습니다**.

### 비교: 다중 분류 vs 다중 레이블 분류

| 구분 | 다중 분류 (Multi-class) | 다중 레이블 분류 (Multi-label) |
| :--- | :--- | :--- |
| **정의** | 여러 개 중 **딱 하나**만 선택 | 여러 개 중 **여러 개** 선택 가능 |
| **예시** | 이 동물은? (고양이, 개, 새 중 택1) | 이 영화 장르는? (액션, SF, 로맨스 다수 포함 가능) |
| **정답 형태** | `y = 2` (단일 숫자) | `y = [1, 0, 1]` (원핫/멀티핫 벡터) |
| **마지막 함수** | **Softmax** (전체 합이 1) | **Sigmoid** (각각 독립적으로 0~1 확률) |

**대표적인 활용 사례**:
- 뉴스 기사 태그 달기 (정치, 경제, 사회 동시 포함)
- 블로그 해시태그 추천
- 영화/음악 장르 분류

## 2. 데이터 준비 & MultiLabelBinarizer
다중 레이블 데이터는 정답이 `['Action', 'Sci-Fi']` 처럼 리스트 형태입니다.
컴퓨터가 이해하려면 이를 `[1, 0, 1, 0...]` 처럼 **0과 1의 체크리스트(Multi-hot Vector)**로 바꿔줘야 합니다.

In [ ]:
import pandas as pd

# 1. 가상의 영화 줄거리 데이터
data = pd.DataFrame({
    'plot': [
        "A man fights crime in a futuristic city.",       # 액션, SF
        "A love story set in wartime.",                   # 로맨스, 드라마
        "Aliens invade Earth and a war begins.",          # 액션, SF, 전쟁
        "A detective solves a complicated crime case.",   # 범죄, 미스터리
        "A dramatic romance in the midst of a tragedy."   # 드라마, 로맨스
    ],
    'genres': [
        ['Action', 'Sci-Fi'],
        ['Romance', 'Drama'],
        ['Action', 'Sci-Fi', 'War'],
        ['Crime', 'Mystery'],
        ['Drama', 'Romance']
    ]
})
print("--- 원본 데이터 ---")
display(data)

In [ ]:
# 2. 정답(Label) 전처리: 리스트 -> 멀티핫 벡터
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['genres'])

print("\n--- 변환된 정답 (Multi-hot Vector) ---")
print(y)
print(f"\n정답 클래스 순서: {mlb.classes_}")

# 보기 좋게 데이터프레임으로 확인
label_df = pd.DataFrame(y, columns=mlb.classes_, index=data['plot'])
display(label_df)

In [ ]:
# 3. 입력(Input) 전처리: 텍스트 -> 숫자 벡터 (TF-IDF)
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['plot'])

# 데이터 확인
print(f"데이터 크기: {X.shape} (문서 5개, 단어 {X.shape[1]}개)")

## 3. 다중 레이블 분류 모델 학습
기존 분류기는 정답을 하나만 고르려고 합니다. 다중 레이블을 풀려면 전략이 필요합니다.

### 전략: One-vs-Rest (OvR)
- **"액션이냐 아니냐?"**, **"SF냐 아니냐?"**, **"로맨스냐 아니냐?"** ...
- 이렇게 모든 장르마다 **O/X 퀴즈(이진 분류기)**를 따로따로 만들어서 풉니다.
- `MultiOutputClassifier` 또는 `OneVsRestClassifier`를 사용하면 자동으로 해줍니다.

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀(기본 분류기)를 여러 개 묶어서 동시에 풉니다.
clf = MultiOutputClassifier(LogisticRegression())
clf.fit(X, y)
print("학습 완료!")

In [ ]:
# 4. 예측해보기
test_plot = ["A space war with aliens"] # SF, Action, War 예상

# 전처리 (Transform)
X_test = vectorizer.transform(test_plot)

# 예측
pred_matrix = clf.predict(X_test)
print(f"예측된 벡터: {pred_matrix}")

# 사람이 보기 좋게 다시 텍스트로 변환 (Inverse Transform)
pred_labels = mlb.inverse_transform(pred_matrix)
print(f"예측된 장르: {pred_labels}")

# 데이터가 너무 적어서 정확하지 않을 수 있습니다 :)
# 하지만 'War', 'Action' 등이 나올 확률이 높습니다.

## (참고) 딥러닝에서는?
딥러닝으로 다중 레이블 분류를 할 때는:
1. **출력층 노드 수**: 클래스 개수만큼 (예: 장르가 10개면 10개)
2. **활성화 함수**: `Sigmoid` (각 노드가 0~1 사이 확률을 독립적으로 가짐)
    - Softmax는 다 합쳐서 1이 되어야 하므로 쓰지 않습니다.
3. **손실 함수**: `Binary Cross Entropy` (각 노드별로 정답/오답 채점)